In [1]:
import pandas as pd
import numpy as np

### 1.（15%）读取数据集data.csv，进⾏数据预处理。

#### Q1.（5%）选取问卷中的SC155Q01HA,SC155Q02HA,SC155Q03HA,SC155Q04HA,SC155Q05HA5个离散性特征作为特征集，分别介绍这些特征所代表的含义和各⾃取值范围，注意到这些特征名称本⾝较为冗⻓且不易于理解，请对特征名进行简化修改，并删除存在缺失值的行。
含义和取值范围见实验报告。


In [2]:
file_path = 'data.csv'
df = pd.read_csv(file_path, index_col = 0)
target = ['SC155Q01HA', 'SC155Q02HA', 'SC155Q03HA', 'SC155Q04HA', 'SC155Q05HA']
data = df[target]
data_drop = data.dropna()
rename = ['1', '2', '3', '4', '5']
data_drop.columns = rename
print(data_drop)


         1    2    3    4    5
1      2.0  2.0  2.0  1.0  1.0
2      2.0  2.0  2.0  2.0  2.0
3      2.0  3.0  1.0  1.0  1.0
4      2.0  3.0  2.0  2.0  2.0
5      2.0  4.0  2.0  2.0  2.0
...    ...  ...  ...  ...  ...
21899  2.0  3.0  2.0  2.0  2.0
21900  3.0  3.0  2.0  2.0  2.0
21901  3.0  3.0  3.0  2.0  2.0
21902  4.0  3.0  3.0  3.0  3.0
21903  3.0  3.0  2.0  2.0  2.0

[20681 rows x 5 columns]


#### Q2.（10%）注意到选取的特征可能存在相同取值（如特征A和B都可能取值0），不便于后续的关联分析过程。请构建项集索引，并依据索引内容进行特征值替换。项集索引字典形式如下：


In [3]:
ind2val = {}
crt_idx = 0

for i in rename:
    for j in range(1, 5):
        ind2val[crt_idx] = f'[{i}]=[{j}]'
        crt_idx += 1
data_replace = data_drop.copy()

i = 0
for col in data_drop.columns:
    for value in range(1, 5):
        data_replace[col]= data_replace[col].replace(value, value+i-1)
    i+=4
print(data_replace)

         1    2     3     4     5
1      1.0  5.0   9.0  12.0  16.0
2      1.0  5.0   9.0  13.0  17.0
3      1.0  6.0   8.0  12.0  16.0
4      1.0  6.0   9.0  13.0  17.0
5      1.0  7.0   9.0  13.0  17.0
...    ...  ...   ...   ...   ...
21899  1.0  6.0   9.0  13.0  17.0
21900  2.0  6.0   9.0  13.0  17.0
21901  2.0  6.0  10.0  13.0  17.0
21902  3.0  6.0  10.0  14.0  18.0
21903  2.0  6.0   9.0  13.0  17.0

[20681 rows x 5 columns]


### 2.（60%）基于预处理后的数据集，编写算法代码进行频繁项集挖掘。

#### Q1.（30%）请参考以下 Apriori 产⽣频繁项集的算法流程，⾃⾏编写相应代码，分别以最⼩⽀持度阈值为0.25和0.5，挖掘频繁项集。


In [4]:
def Apriori(data, min_support=0.5):
    result = {}
    col_num = len(data.columns)
    val_dict = {}
    #找到各列可能取值
    for col in data.columns:
        val_dict[col] = data[col].unique().tolist()
    #计算频繁1项集
    r1 = {}
    for col in data.columns:
        r1[col] = {}
        for v in val_dict[col]:
            freq = len(data[data[col] == v])/len(data)
            if freq >= min_support:
                r1[col][(v,)] = (freq,)
        result[1] = r1
    #计算频繁k项集
    for i in range(2, col_num+1):
        if (i-1 not in result or  len(result[i-1]) == 1):
            break
        r = {}
        for col1 in result[i-1].keys():
            for col2 in result[i-1].keys():
                #计算候选k项集
                if col1[:-1] == col2[:-1] and col1[-1] < col2[-1]:
                    new_col = col1+col2[-1]
                    #用于存储可能的取值
                    test_val = []
                    for v1 in result[i-1][col1].keys():
                        for v2 in result[i-1][col2].keys():
                            if v1[:-1] == v2[:-1]:
                                new_val = (v1)+(v2[-1],)
                                test_val.append(new_val)
                    if len(test_val) == 0:
                        continue
                    #计算实际的频率
                    actual_val = []
                    actual_freq = []
                    for v in test_val:
                        a = data.copy()
                        for j in range(len(new_col)):
                            a = a[a[new_col[j]]==v[j]]
                        freq = len(a)/len(data)
                        if freq >= min_support:
                            actual_val.append(v)
                            actual_freq.append(freq)
                    r[new_col] = {tuple(actual_val[i]): (actual_freq[i],) for i in range(len(actual_val))}
        if r:   
            result[i] = r
    return result
print(Apriori(data_replace, 0.25))
print(Apriori(data_replace, 0.5))
        

{1: {'1': {(1.0,): (0.26816885063584933,), (2.0,): (0.3985784052995503,)}, '2': {(6.0,): (0.4188385474590204,), (7.0,): (0.3350901793917122,)}, '3': {(9.0,): (0.3363473719839466,), (10.0,): (0.3700014506068372,)}, '4': {(13.0,): (0.29447318795029254,), (14.0,): (0.4361491223828635,)}, '5': {(17.0,): (0.27334268168850634,), (18.0,): (0.48063439872346597,)}}, 2: {'12': {(2.0, 6.0): (0.2602872201537643,)}, '13': {(2.0, 10.0): (0.2744064600357816,)}, '14': {(2.0, 14.0): (0.26183453411343743,)}, '15': {(2.0, 18.0): (0.27401963154586334,)}, '23': {}, '24': {(6.0, 14.0): (0.2694260432280837,)}, '25': {(6.0, 18.0): (0.2733910352497461,)}, '34': {(10.0, 14.0): (0.27450316715826123,)}, '35': {(10.0, 18.0): (0.2713118321164354,)}, '45': {(14.0, 18.0): (0.33054494463517237,)}}, 3: {'123': {}, '124': {}, '125': {}, '134': {}, '135': {}, '145': {}, '245': {}, '345': {}}}
{1: {'1': {}, '2': {}, '3': {}, '4': {}, '5': {}}}


#### Q2.（15%）当最小支持度为0.5时，频繁项集数量较少。请将各特征原始取值为1和2的单元格统一修改其值为0，取值为3和4的单元格统一修改其值为1。重复T1-Q2的项集索引构建过程，并以最小支持度阈值为0.5，挖掘频繁项集。

In [5]:
data_rereplace = data_drop.copy()
for col in data_rereplace.columns:
    data_rereplace[col]= data_rereplace[col].replace(1, 0)
    data_rereplace[col]= data_rereplace[col].replace(2, 0)
    data_rereplace[col]= data_rereplace[col].replace(3, 1)
    data_rereplace[col]= data_rereplace[col].replace(4, 1)
print('更新前'+'-'*20)
print(data_rereplace)

for i in rename:
    for j in range(0, 2):
        ind2val[crt_idx] = f'[{i}]=[{j}]'
        crt_idx += 1

i = 0
for col in data_drop.columns:
    for value in data_drop[col].unique():
        data_rereplace[col]= data_rereplace[col].replace(value, value+i)
    i+=2
print('更新后'+'-'*20)
print(data_rereplace)
print('频繁项集'+'-'*20)
print(Apriori(data_rereplace, 0.5))

更新前--------------------
         1    2    3    4    5
1      0.0  0.0  0.0  0.0  0.0
2      0.0  0.0  0.0  0.0  0.0
3      0.0  1.0  0.0  0.0  0.0
4      0.0  1.0  0.0  0.0  0.0
5      0.0  1.0  0.0  0.0  0.0
...    ...  ...  ...  ...  ...
21899  0.0  1.0  0.0  0.0  0.0
21900  1.0  1.0  0.0  0.0  0.0
21901  1.0  1.0  1.0  0.0  0.0
21902  1.0  1.0  1.0  1.0  1.0
21903  1.0  1.0  0.0  0.0  0.0

[20681 rows x 5 columns]
更新后--------------------
         1    2    3    4    5
1      0.0  0.0  0.0  0.0  0.0
2      0.0  0.0  0.0  0.0  0.0
3      0.0  3.0  0.0  0.0  0.0
4      0.0  3.0  0.0  0.0  0.0
5      0.0  3.0  0.0  0.0  0.0
...    ...  ...  ...  ...  ...
21899  0.0  3.0  0.0  0.0  0.0
21900  1.0  3.0  0.0  0.0  0.0
21901  1.0  3.0  5.0  0.0  0.0
21902  1.0  3.0  5.0  7.0  9.0
21903  1.0  3.0  0.0  0.0  0.0

[20681 rows x 5 columns]
频繁项集--------------------
{1: {'1': {(1.0,): (0.625211546830424,)}, '2': {(3.0,): (0.6224553938397563,)}, '3': {(5.0,): (0.547507373918089,)}, '4': {(7.0,): 

#### Q3.（15%）分析Q1和Q2的结果，你有什么发现？请根据各特征定义，分析产⽣这种情况的原因。
见实验报告

### 3.（25%）基于T2-Q2得到的频繁项集挖掘结果，编写算法代码进⾏关联规则提取。

#### Q1.（15%）请以最⼩置信度阈值为0.8，提取形如X->{1}的关联规则，并输出它们的置信度和提升度。

In [6]:
def find_rule(data, confidence=0.8):
    d1 = data[1]
    d2 = data[2]
    for col, val_freq in d2.items():
        for val, freq in val_freq.items():
            for i in range(len(col)):
                target = col[i]
                condition = col[:i] + col[i+1:]
                target_freq = d1[target][(val[i],)]
                condition_freq = d1[condition][(val[(i+1)%2],)]
                print(f'关联规则: {val[i]} -> {val[(i+1)%2]},置信度: {freq[0]/target_freq[0]}, 提升度: {freq[0]/(target_freq[0]*condition_freq[0])}')
find_rule(Apriori(data_rereplace, 0.5))


关联规则: 1.0 -> 3.0,置信度: 0.8226604795050271, 提升度: 1.3216376428682872
关联规则: 3.0 -> 1.0,置信度: 0.8263031150469975, 提升度: 1.3216376428682872
关联规则: 1.0 -> 5.0,置信度: 0.8125290023201858, 提升度: 1.4840512494024343
关联规则: 5.0 -> 1.0,置信度: 0.9278459772145193, 提升度: 1.4840512494024343
关联规则: 1.0 -> 7.0,置信度: 0.8010054137664347, 提升度: 1.3423217698811796
关联规则: 7.0 -> 1.0,置信度: 0.8392350700915647, 提升度: 1.3423217698811796
关联规则: 1.0 -> 9.0,置信度: 0.8194895591647333, 提升度: 1.2957082242420377
关联规则: 9.0 -> 1.0,置信度: 0.8100917431192662, 提升度: 1.2957082242420377
关联规则: 7.0 -> 9.0,置信度: 0.8674337573940524, 提升度: 1.3715135731396328
关联规则: 9.0 -> 7.0,置信度: 0.8184250764525994, 提升度: 1.3715135731396328


#### Q2.（10%）参考项集索引的对应关系，对以上频繁项集和关联规则结果进⾏简要分析和总结。
见实验报告